# Clase Práctica 01

# Neural Network - Redes Neuronales Artificiales

# Primera Red Neuronal con Keras



# Introducción

Este tutorial muestra el flujo de trabajo básico de usar Keras con un modelo lineal simple, se va a explicar y desarrollar un código básico con Tensorflow y Keras.

Los pasos a seguir para el desarrollo del código son los siguientes: 

1. Cargar los datos.
2. Definir nuestro modelo.
3. Compilar nuestro modelo.
4. Ajustar nuestro modelo.
5. Evaluar nuestro modelo.
6. Unir todo el proceso.



# Base de datos: Pima Indians

En este tutorial vamos a utilizar la base de datos de diabetes Pima Indians. La base de datos describe los datos de los registros médicos de los pacientes que tuvieron un inicio de diabetes dentro de los cinco años. Corresponde a un problema de clasificación binaria (aparición de diabetes como 1 o no aparición como 0). Las variables de entrada que describen a cada paciente son numéricas y tienen escalas variables. La base de datos posee 768 instancias. 

A continuación se enumeran los ocho atributos (descritos en inglés) para el conjunto de datos:

1. Number of times pregnant.
2. Plasma glucose concentration a 2 hours in an oral glucose tolerance test.
3. Diastolic blood pressure (mm Hg).
4. Triceps skin fold thickness (mm).
5. 2-Hour serum insulin (mu U/ml).
6. Body mass index.
7. Diabetes pedigree function.
8. Age (years).
9. Class, onset of diabetes within five years.



# Cargar librerías y datos  

Se utiliza una semilla (seed) para hacer los experimentos de Deep Learning repetibles. Esto es útil si necesita demostrar un resultado, comparar algoritmos usando la misma fuente de aleatoriedad o depurar una parte de su código. Puede inicializar el generador de números aleatorios con cualquier semilla que desee, por ejemplo: 7 como el que usamos en el ejemplo. 

In [33]:
# librerías
import torch
from torch import nn
from torch.nn import Module
from torch.nn import Linear
from torch.nn import Sequential
import numpy

# seed fija para experimentos
seed = 7
numpy.random.seed(seed)

Ahora podemos cargar nuestra base de datos. Se puede cargar el archivo directamente usando la función NumPy loadtxt(). Hay ocho variables de entrada y una variable de salida (la última columna contiene las etiquetas). Una vez cargado, podemos dividir el conjunto de datos en variables de entrada (X) y la variable de clase de salida (Y).

In [19]:
# cargar los datos
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# separamos en variables de entrada (X) y salidas (Y) 
X = dataset[:,0:8]
Y = dataset[:,8]
print("Entradas:")
print(X)
print("Salidas:")
print(Y)
print("Observe que hay 768 ejemplos y 8 atributos:")
print((X.shape))
print("Observe que hay 768 etiquetas")
print((Y.shape))

Entradas:
[[  6.    148.     72.    ...  33.6     0.627  50.   ]
 [  1.     85.     66.    ...  26.6     0.351  31.   ]
 [  8.    183.     64.    ...  23.3     0.672  32.   ]
 ...
 [  5.    121.     72.    ...  26.2     0.245  30.   ]
 [  1.    126.     60.    ...  30.1     0.349  47.   ]
 [  1.     93.     70.    ...  30.4     0.315  23.   ]]
Salidas:
[1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1.
 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0.
 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0.
 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0.
 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0.
 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0.
 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 

# Definir el modelo de red neuronal

Los modelos en Keras se definen como una secuencia de capas. Por lo tanto, vamos a crear un modelo secuencial y agregamos capas secuencialmente para obtener nuestra topología de red. Debemos asegurarnos que la capa de entrada que hemos definimos tenga el número correcto de entradas. Esto se puede especificar al crear la primera capa con el argumento llamado "dimensión de entrada" y establecerlo en 8 para las 8 variables de entrada.

¿Cómo sabemos el número de capas a usar y sus tipos? Esta es una pregunta muy difícil en Deep Learning. Podemos utilizar heurísticas y, a menudo, la mejor estructura de red se encuentra a través de un proceso de experimentación de prueba y error. En general, necesita una red lo suficientemente grande como para capturar la estructura del problema, si es que eso ayuda. En este ejemplo, utilizaremos una estructura de red totalmente conectada (fully connected) con tres capas.

Las capas totalmente conectadas (fully connected) se definen utilizando la clase "Dense". Podemos especificar el número de neuronas en la capa en el primer argumento; el método de inicialización de la red en el segundo argumento como init; y además, se puede especificar la función de activación utilizando el tercer argumento (función de activación). Para nuestro ejemplo, vamos a inicializar los pesos de la red de manera aleatoria, generada a partir de una distribución uniforme. Usaremos en este caso, valores entre 0 y 0.05, porque esa es la inicialización de pesos uniformes que trae de forma predeterminada Keras. Otra alternativa tradicional sería normal para los pequeños números aleatorios generados a partir de una distribución gaussiana.

Usaremos la función de activación de rectificación (ReLU) en las dos primeras capas y la función de activación sigmoide en la capa de salida. Tambien es posible usar las funciones de activación sigmoide y tanh para todas las capas. Actualente, se observa un mejor rendimiento (empírico) utilizando la función de activación de rectificación ReLU. Utilizamos una función de activación sigmoidea en la capa de salida para garantizar que nuestra salida de red esté entre 0 y 1, y sea fácil de asignar a una probabilidad. La primera capa oculta tiene 12 neuronas y espera 8 variables de entrada. La segunda capa oculta tiene 8 neuronas y solo la capa de salida tiene 1 neurona para predecir la clase (aparición de diabetes o no).

<img src="./images_tutoriales/Imagen1.png">


In [20]:
help(Linear)

Help on class Linear in module torch.nn.modules.linear:

class Linear(torch.nn.modules.module.Module)
 |  Linear(in_features: int, out_features: int, bias: bool = True, device=None, dtype=None) -> None
 |  
 |  Applies a linear transformation to the incoming data: :math:`y = xA^T + b`
 |  
 |  This module supports :ref:`TensorFloat32<tf32_on_ampere>`.
 |  
 |  Args:
 |      in_features: size of each input sample
 |      out_features: size of each output sample
 |      bias: If set to ``False``, the layer will not learn an additive bias.
 |          Default: ``True``
 |  
 |  Shape:
 |      - Input: :math:`(*, H_{in})` where :math:`*` means any number of
 |        dimensions including none and :math:`H_{in} = \text{in\_features}`.
 |      - Output: :math:`(*, H_{out})` where all but the last dimension
 |        are the same shape as the input and :math:`H_{out} = \text{out\_features}`.
 |  
 |  Attributes:
 |      weight: the learnable weights of the module of shape
 |          :math:`(

In [31]:
# Las redes (modelos en general) deben heredar desde torch.nn.Module
# modelo secuencial
class FFNN(Module):
  
  #definición 
  def __init__(self, d0=8, d1=12):
    super(FFNN, self).__init__()
    
    # Usa 'capas lineales' en vez de parámetros explícitos
    # Los parámetros se incializan automáticamente y se agregan
    # a los parámetros de la red
    self.fc_1 = Linear(d0,d1)#8 entradas y 12 neuronas en capa oculta
    self.fc_2 = Linear(d1,d1) # 8 neuronas en capa oculta 
    self.fc_out = Linear(d1,1)# 1 neurona en la salida
    
  def forward(self, x):
    h1 = nn.ReLU(self.fc_1(x))
    h2 = nn.ReLU(self.fc_2(h1))
    y_pred = nn.ReLU(self.fc_out(h2))
    
    return y_pred

# Compilar el modelo de red neuronal

Ahora que el modelo está definido, podemos compilarlo. La compilación del modelo utiliza las bibliotecas numéricas eficientes llamadas backend, como TensorFlow o Theano. El backend elige automáticamente la mejor forma de representar la red para entrenarse y hacer predicciones para ejecutar en su hardware. Al compilar, debemos especificar algunas propiedades adicionales requeridas al entrenar la red. Recuerde que entrenar una red significa encontrar el mejor conjunto de pesos y sesgos para hacer predicciones para este problema.

Debemos especificar la función de pérdida (loss) que se usará para evaluar un conjunto de entrenamiento; tambien se debe especificar el optimizador para buscar los pesos óptimos de la red; y cualquier métrica que nos permita evaluar el desempeño  durante el entrenamiento. En este caso utilizaremos la pérdida logarítmica (logarithmic loss), que para un problema de **clasificación binaria** se define en Keras como **crosentropia binaria (binary_crossentropy)**. También utilizaremos el algoritmo de **descenso de gradiente** eficiente llamado **adam**. Finalmente, debido a que es un problema de clasificación, la métrica que vamos a recopilar e informar es la **precisión (accuracy)**.

In [7]:
# Compilar modelo
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters())

# Ajustar el modelo de red neuronal (Fit)

Hemos definido nuestro modelo y lo hemos compilado listo para poder realizar un cálculo eficiente. Ahora es el momento de ejecutar el modelo en algunos datos. Podemos entrenar nuestro modelo con nuestros datos cargados llamando a la función de ajuste **fit ()** en el modelo.

El proceso de entrenamiento se ejecutará para un número fijo de iteraciones a través del conjunto de datos llamado **épocas (epochs)**, que debemos especificar usando el argumento nb_epoch. También podemos establecer el número de instancias que se evalúan antes de que se realice una actualización de peso en la red, denominada **tamaño de lote (batch_size)** que es establecido con el argumento de **batch_size**. Para este problema ejecutaremos un pequeño número de épocas (150) y usaremos un tamaño de lote relativamente pequeño de 10. Una vez más, estos pueden ser seleccionados experimentalmente por prueba y error.

In [9]:
# Ajustar el modelo
epochs=150
batch_size=10
train_loader=X
target = Y


 
# Train the network
 
for epoch in range(epochs):
    for batch, (data, target) in enumerate(train_loader):
        # Obtaining the cuda parameters
        data = data.to(device=device)
        target = target.to(device=device)
       
        # Reshaping to suit our model
        data = data.reshape(data.shape[0], -1)
       
        # Forward propagation
        score = model(data)
        loss = criterion(score, target)
       
        # Backward propagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Epoch 1/150
77/77 [==============================] - 5s 8ms/step - loss: 0.6797 - accuracy: 0.6432
Epoch 2/150
77/77 [==============================] - 0s 6ms/step - loss: 0.6631 - accuracy: 0.6510
Epoch 3/150
77/77 [==============================] - 0s 4ms/step - loss: 0.6589 - accuracy: 0.6497
Epoch 4/150
77/77 [==============================] - 0s 4ms/step - loss: 0.6521 - accuracy: 0.6497
Epoch 5/150
77/77 [==============================] - 0s 4ms/step - loss: 0.6448 - accuracy: 0.6549
Epoch 6/150
77/77 [==============================] - 0s 3ms/step - loss: 0.6315 - accuracy: 0.6693
Epoch 7/150
77/77 [==============================] - 0s 3ms/step - loss: 0.6226 - accuracy: 0.6602
Epoch 8/150
77/77 [==============================] - 0s 3ms/step - loss: 0.6079 - accuracy: 0.6771
Epoch 9/150
77/77 [==============================] - 0s 3ms/step - loss: 0.5985 - accuracy: 0.6940
Epoch 10/150
77/77 [==============================] - 0s 4ms/step - loss: 0.5951 - accuracy: 0.6940
Epoch 11/

# Evaluar el modelo de red neuronal 

Hemos entrenado nuestra red neuronal en **todo** el conjunto de datos y podemos evaluar el rendimiento de la red con el mismo conjunto de datos. Esto no se suele hacer, sino que esto solo **nos dará una idea de qué tan bien hemos modelado el conjunto de datos (por ejemplo, la precisión del entrenamiento), pero no tenemos idea de qué tan bien podría funcionar el algoritmo con los nuevos datos**. Lo hemos hecho para simplificar, pero lo ideal es que pueda separar sus datos en conjuntos de datos de entrenamiento y test, uno para el entrenamiento y otro para la evaluación de su modelo.

Puede evaluar su modelo en sus datos de entrenamiento utilizando la función evaluate() de su modelo y pasarle la misma entrada y salida utilizada para entrenar el modelo. Esto generará una predicción para cada par de entrada y salida, y recopilará el rendimiento de la red, incluida la pérdida promedio y cualquier métrica que haya configurado, como la precisión.

In [11]:
# evaluar el modelo
scores = model.evaluate(X, Y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

24/24 [==============================] - 0s 3ms/step - loss: 0.4304 - accuracy: 0.7865
accuracy: 78.65%


# Unir todo el proceso

Acaba de ver cómo puede crear fácilmente su primer modelo de red neuronal en Keras. Vamos a unirlo todo en un ejemplo de código completo.

In [34]:
from keras.models import Sequential
from keras.layers import Dense
import numpy

seed = 7
numpy.random.seed(seed)
# datos
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# entradas y salidas
X = dataset[:,0:8]
Y = dataset[:,8]
# modelo
model = Sequential()
model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
model.add(Dense(8, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))
# compilar
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# ajustar
model.fit(X, Y, nb_epoch=150, batch_size=10)
# evaluar
scores = model.evaluate(X, Y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

TypeError: ('Keyword argument not understood:', 'init')